In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from warnings import filterwarnings
from mpl_toolkits.mplot3d import Axes3D
import statsmodels.api as sm
import missingno as msno
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.neighbors import LocalOutlierFactor
from scipy.stats import levene
from scipy.stats import shapiro
from scipy.stats.stats import pearsonr
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.preprocessing import scale
from sklearn.model_selection import ShuffleSplit, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import model_selection
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
import xgboost as xgb
from xgboost import XGBRegressor, XGBClassifier
from lightgbm import LGBMRegressor, LGBMClassifier
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import tree
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score, roc_curve

In [ ]:
filterwarnings("ignore")

# DATA SOURCE

In [ ]:
Pulsar = pd.read_csv("../input/pulsar-classification-for-class-prediction/Pulsar.csv")

In [ ]:
# SAVING
data = Pulsar.copy()

# VISUALIZATION
dataV = data.copy()
dataV["Class"] = pd.Categorical(dataV["Class"])

# ADDITIONAL 
df = data.select_dtypes(include=["float64","int64","int32"])

# INFORMATIONS

In [ ]:
print(data.shape)

In [ ]:
print(data.columns)

In [ ]:
print(data.info())

In [ ]:
print(data.describe().T)

In [ ]:
print(df.corr())

In [ ]:
print(data.groupby(["Class"])["Mean_Integrated"].mean())

In [ ]:
print(data.groupby(["Class"])["SD"].mean())

In [ ]:
print(data.groupby(["Class"])["EK"].mean())

In [ ]:
print(data.groupby(["Class"])["Skewness"].mean())

In [ ]:
print(data.groupby(["Class"])["Mean_DMSNR_Curve"].mean())

In [ ]:
print(data.groupby(["Class"])["SD_DMSNR_Curve"].mean())

In [ ]:
print(data.groupby(["Class"])["EK_DMSNR_Curve"].mean())

In [ ]:
print(data.groupby(["Class"])["Skewness_DMSNR_Curve"].mean())

In [ ]:
print(data.where(data["Mean_Integrated"] > 130).value_counts()) # 3546 OBSERVATIONS

In [ ]:
print(data.where(data["SD"] > 50).value_counts()) # 5530 OBSERVATIONS

In [ ]:
print(data.where(data["SD_DMSNR_Curve"] > 30).value_counts()) # 4131 OBSERVATIONS

In [ ]:
print(data.isnull().sum()) # No missing value

# CORRELATION - NORMALITY - HOMOGENEITY

### CORRELATION

In [ ]:
corrPearson = data.corr(method="pearson")

In [ ]:
corrSpearman = data.corr(method="spearman")

In [ ]:
figure = plt.figure(figsize=(20,10))
sns.heatmap(corrPearson,annot=True,cmap="RdYlGn",vmin=-1,vmax=1,center=0,linecolor="black")
plt.show()

In [ ]:
figure = plt.figure(figsize=(20,10))
sns.heatmap(corrSpearman,annot=True,cmap="RdYlGn",vmin=-1,vmax=1,center=0,linecolor="black")
plt.show()

### NORMALITY

In [ ]:
for i in data.columns:
    print("----"*30)
    print(i)
    print("%.3f - %.3f" % shapiro(data[i]))

### HOMOGENEITY

In [ ]:
print("%.3f - %.3f" % levene(data["Mean_Integrated"],data["SD"],data["EK"],
                            data["Skewness"],data["Mean_DMSNR_Curve"],data["SD_DMSNR_Curve"],
                            data["EK_DMSNR_Curve"],data["Skewness_DMSNR_Curve"]))

# VISUALIZATION

### HIST

In [ ]:
data.hist(figsize=(20,20))
plt.show()

### BOX

In [ ]:
figure = plt.figure(figsize=(20,8))
sns.boxplot(x="Mean_Integrated",y="Class",data=dataV)
plt.show()

In [ ]:
figure = plt.figure(figsize=(20,8))
sns.boxplot(x="SD",y="Class",data=dataV)
plt.show()

In [ ]:
figure = plt.figure(figsize=(20,8))
sns.boxplot(x="EK",y="Class",data=dataV)
plt.show()

In [ ]:
figure = plt.figure(figsize=(20,8))
sns.boxplot(x="Skewness",y="Class",data=dataV)
plt.show()

In [ ]:
figure = plt.figure(figsize=(20,8))
sns.boxplot(x="Mean_DMSNR_Curve",y="Class",data=dataV)
plt.show()

In [ ]:
figure = plt.figure(figsize=(20,8))
sns.boxplot(x="SD_DMSNR_Curve",y="Class",data=dataV)
plt.show()

In [ ]:
figure = plt.figure(figsize=(20,8))
sns.boxplot(x="EK_DMSNR_Curve",y="Class",data=dataV)
plt.show()

In [ ]:
figure = plt.figure(figsize=(20,8))
sns.boxplot(x="Skewness_DMSNR_Curve",y="Class",data=dataV)
plt.show()

### LINE

In [ ]:
figure = plt.figure(figsize=(20,8))
sns.lineplot(x="SD",y="Mean_Integrated",hue="Class",data=dataV)
plt.show()

In [ ]:
figure = plt.figure(figsize=(20,8))
sns.lineplot(x="EK",y="Mean_Integrated",hue="Class",data=dataV)
plt.show()

In [ ]:
figure = plt.figure(figsize=(20,8))
sns.lineplot(x="Skewness",y="Mean_Integrated",hue="Class",data=dataV)
plt.show()

In [ ]:
figure = plt.figure(figsize=(20,8))
sns.lineplot(x="SD_DMSNR_Curve",y="Mean_Integrated",hue="Class",data=dataV)
plt.show()

In [ ]:
figure = plt.figure(figsize=(20,8))
sns.lineplot(x="EK_DMSNR_Curve",y="Mean_Integrated",hue="Class",data=dataV)
plt.show()

In [ ]:
figure = plt.figure(figsize=(20,8))
sns.lineplot(x="Skewness_DMSNR_Curve",y="Mean_Integrated",hue="Class",data=dataV)
plt.show()

### HISTPLOT

In [ ]:
figure = plt.figure(figsize=(20,8))
sns.histplot(
    data,
    x="Mean_Integrated", hue="Class",
    multiple="stack",
    palette="light:m_r",
    edgecolor=".3",
    linewidth=.5,
    log_scale=True,
)

In [ ]:
figure = plt.figure(figsize=(20,8))
sns.histplot(
    data,
    x="SD", hue="Class",
    multiple="stack",
    palette="light:m_r",
    edgecolor=".3",
    linewidth=.5,
    log_scale=True,
)

In [ ]:
figure = plt.figure(figsize=(20,8))
sns.histplot(
    data,
    x="EK", hue="Class",
    multiple="stack",
    palette="light:m_r",
    edgecolor=".3",
    linewidth=.5,
    log_scale=True,
)

In [ ]:
figure = plt.figure(figsize=(20,8))
sns.histplot(
    data,
    x="Skewness", hue="Class",
    multiple="stack",
    palette="light:m_r",
    edgecolor=".3",
    linewidth=.5,
    log_scale=True,
)

In [ ]:
figure = plt.figure(figsize=(20,8))
sns.histplot(
    data,
    x="Mean_DMSNR_Curve", hue="Class",
    multiple="stack",
    palette="light:m_r",
    edgecolor=".3",
    linewidth=.5,
    log_scale=True,
)

In [ ]:
figure = plt.figure(figsize=(20,8))
sns.histplot(
    data,
    x="SD_DMSNR_Curve", hue="Class",
    multiple="stack",
    palette="light:m_r",
    edgecolor=".3",
    linewidth=.5,
    log_scale=True,
)

In [ ]:
figure = plt.figure(figsize=(20,8))
sns.histplot(
    data,
    x="EK_DMSNR_Curve", hue="Class",
    multiple="stack",
    palette="light:m_r",
    edgecolor=".3",
    linewidth=.5,
    log_scale=True,
)

In [ ]:
figure = plt.figure(figsize=(20,8))
sns.histplot(
    data,
    x="Skewness_DMSNR_Curve", hue="Class",
    multiple="stack",
    palette="light:m_r",
    edgecolor=".3",
    linewidth=.5,
    log_scale=True,
)

### JOINTPLOT

In [ ]:
figure = plt.figure(figsize=(20,8))
sns.jointplot(x="SD",y="Mean_DMSNR_Curve", color="#4CB391",hue="Class",data=data)
plt.show()

In [ ]:
figure = plt.figure(figsize=(20,8))
sns.jointplot(x="SD",y="EK_DMSNR_Curve", color="#4CB391",hue="Class",data=data)
plt.show()

In [ ]:
figure = plt.figure(figsize=(20,8))
sns.jointplot(x="SD",y="Skewness_DMSNR_Curve", color="#4CB391",hue="Class",data=data)
plt.show()

### JOINTGRID

In [ ]:
figure = plt.figure(figsize=(20,8))
g = sns.JointGrid(data=data, x="SD", y="EK", space=0)
g.plot_joint(sns.kdeplot,cmap="rocket")
plt.show()

In [ ]:
figure = plt.figure(figsize=(20,8))
g = sns.JointGrid(data=data, x="SD_DMSNR_Curve", y="EK_DMSNR_Curve", space=0)
g.plot_joint(sns.kdeplot,cmap="rocket")
plt.show()

### 3D

In [ ]:
fig = plt.figure(figsize=(20,10))
ax = Axes3D(fig)
ax.scatter(data["SD_DMSNR_Curve"], data["EK_DMSNR_Curve"], data["Skewness_DMSNR_Curve"], c="red", s=20, alpha=0.2)
plt.show()

In [ ]:
fig = plt.figure(figsize=(20,10))
ax = Axes3D(fig)
ax.scatter(data["SD"], data["EK"], data["Skewness"], c="black", s=20, alpha=0.2)
plt.show()

In [ ]:
fig = plt.figure(figsize=(20,10))
ax = Axes3D(fig)
ax.scatter(data["SD"], data["EK"], data["Mean_Integrated"], c="green", s=20, alpha=0.2)
plt.show()

# AGAINST VALUES

In [ ]:
DataForA = data.copy()

In [ ]:
clf = LocalOutlierFactor()
clf.fit_predict(DataForA)

In [ ]:
score = clf.negative_outlier_factor_
sortedscore = np.sort(score)
print(sortedscore[0:50])

# It seems there is no against value

# MODEL PROCESS FOR CLASS CLASSIFICATION

### X & Y FOR MODELS

In [ ]:
x = data.drop("Class",axis=1)
y = data["Class"]

xTrain,xTest,yTrain,yTest = train_test_split(x,y,test_size=0.2,random_state=42)

### MODELS

In [ ]:
lj = LogisticRegression(solver="liblinear").fit(xTrain,yTrain)
gnb = GaussianNB().fit(xTrain,yTrain)
knnc = KNeighborsClassifier().fit(xTrain,yTrain)
cartc = DecisionTreeClassifier(random_state=42).fit(xTrain,yTrain)
rfc = RandomForestClassifier(random_state=42,verbose=False).fit(xTrain,yTrain)
gbmc = GradientBoostingClassifier(verbose=False).fit(xTrain,yTrain)
xgbc = XGBClassifier().fit(xTrain,yTrain)
lgbmc = LGBMClassifier().fit(xTrain,yTrain)
catbc = CatBoostClassifier(verbose=False).fit(xTrain,yTrain)

### COMPARISON

In [ ]:
modelsc = [lj,gnb,knnc,cartc,rfc,gbmc,xgbc,lgbmc,catbc]

In [ ]:
for model in modelsc:
    name = model.__class__.__name__
    predict = model.predict(xTest)
    R2CV = cross_val_score(model,xTest,yTest,cv=10,verbose=False).mean()
    error = -cross_val_score(model,xTest,yTest,cv=10,scoring="neg_mean_squared_error",verbose=False).mean()
    roc = roc_curve(yTest,predict)
    print(name + ": ")
    print("-" * 10)
    print("ACC-->",accuracy_score(yTest,predict))
    print("R2CV-->",R2CV)
    print("MEAN SQUARED ERROR-->",np.sqrt(error))
    print("ROC-->",roc)
    print("-" * 30)

In [ ]:
r = pd.DataFrame(columns=["MODELS","R2CV"])
for model in modelsc:
    name = model.__class__.__name__
    R2CV = cross_val_score(model,xTest,yTest,cv=10,verbose=False).mean()
    result = pd.DataFrame([[name,R2CV*100]],columns=["MODELS","R2CV"])
    r = r.append(result)
    
figure = plt.figure(figsize=(20,8))   
sns.barplot(x="R2CV",y="MODELS",data=r,color="k")
plt.xlabel("R2CV")
plt.ylabel("MODELS")
plt.xlim(0,100)
plt.title("MODEL ACCURACY COMPARISON")
plt.show()

In [ ]:
r = pd.DataFrame(columns=["MODELS","error"])
for model in modelsc:
    name = model.__class__.__name__
    error = -cross_val_score(model,xTest,yTest,cv=10,scoring="neg_mean_squared_error").mean()
    result = pd.DataFrame([[name,np.sqrt(error)]],columns=["MODELS","error"])
    r = r.append(result)
    
figure = plt.figure(figsize=(20,8))   
sns.barplot(x="error",y="MODELS",data=r,color="r")
plt.xlabel("ERROR")
plt.ylabel("MODELS")
plt.xlim(0,2)
plt.title("MODEL ERROR COMPARISON")
plt.show()

### ADDITIONAL MODEL ANN

In [ ]:
scaler = StandardScaler().fit(xTrain,yTrain)
xRTrain = scaler.transform(xTrain)
xRTest = scaler.transform(xTest)

In [ ]:
mlpc = MLPClassifier().fit(xRTrain,yTrain)
predict = mlpc.predict(xRTest)

R2CV = cross_val_score(mlpc,xRTest,yTest,cv=10).mean()
print(R2CV)
error = mean_squared_error(yTest,predict)
print(np.sqrt(error))

# BEST MODEL IS XGBClassifier -- > 0.9824